In [8]:
import requests
import itertools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, Point

In [9]:
BASE_URL = 'http://127.0.0.1:5000/route/v1/driving/{};{}?steps=false&geometries=geojson&overview=simplified'

puntos = pd.read_csv('data/coords.csv')
puntos_str = [str(puntos.lon[i]) + ',' + str(puntos.lat[i]) for i in range(puntos.shape[0])]

puntos_str[0:3]

['-56.174030195714,-34.8634731144801',
 '-56.8155472789126,-30.4070624167322',
 '-56.0461527421891,-34.7048062340561']

In [10]:
product = list(itertools.product(puntos_str, puntos_str))
len(product)

361

In [11]:
urls = [BASE_URL.format(i[0], i[1]) for i in product]
urls[0:3]

['http://127.0.0.1:5000/route/v1/driving/-56.174030195714,-34.8634731144801;-56.174030195714,-34.8634731144801?steps=false&geometries=geojson&overview=simplified',
 'http://127.0.0.1:5000/route/v1/driving/-56.174030195714,-34.8634731144801;-56.8155472789126,-30.4070624167322?steps=false&geometries=geojson&overview=simplified',
 'http://127.0.0.1:5000/route/v1/driving/-56.174030195714,-34.8634731144801;-56.0461527421891,-34.7048062340561?steps=false&geometries=geojson&overview=simplified']

In [28]:
lines = []
coords = []

for url in urls:
    response = requests.get(url)
    coord = response.json()['routes'][0]['geometry']['coordinates']
    coord_tuples = [tuple(i) for i in coord]
    points = [Point(i) for i in coord]
    lines.append(LineString(points))
    coords.append(coord_tuples)

id_diadas = list(itertools.product(puntos.DEPTO, puntos.DEPTO))

In [47]:
test = {
    'origen': [],
    'destino': [],
    'coordinates': []
    }

In [48]:
for i in range(25):
    test["ori"].append(id_diadas[i][0])
    test["des"].append(id_diadas[i][1])
    test['coordinates'].append(coords[i])

In [58]:
dict_you_want = {'ori': value for key, value in test.items() if value == 1}

In [59]:
dict_you_want

{}

In [45]:
test = {
    'origen': id_diadas[1][0],
    'destino': id_diadas[1][1],
    'coordinates': coords[1]
    }

In [46]:
test['coordinates']

[(-56.174277, -34.863831),
 (-56.274596, -34.785226),
 (-56.238048, -34.69474),
 (-56.282012, -34.497185),
 (-56.179921, -34.147774),
 (-56.236247, -34.07355),
 (-56.366919, -33.572838),
 (-56.524333, -33.391892),
 (-56.456694, -33.173413),
 (-56.521876, -32.769049),
 (-56.229349, -32.254381),
 (-56.110673, -32.166317),
 (-55.974771, -31.822472),
 (-55.969487, -31.713067),
 (-55.879069, -31.619756),
 (-55.691603, -31.541787),
 (-55.660493, -31.247976),
 (-55.766702, -31.175136),
 (-55.918543, -31.143775),
 (-56.141446, -30.994226),
 (-56.409092, -30.595412),
 (-56.459082, -30.423948),
 (-56.609863, -30.451036),
 (-56.81774, -30.298279),
 (-56.890056, -30.309145),
 (-56.820821, -30.338371),
 (-56.799735, -30.394415)]

In [27]:
tuple(coords[0][0])

(-56.174277, -34.863831)

In [ ]:
ori = pd.DataFrame(id_diadas)[0]
des = pd.DataFrame(id_diadas)[1]

d = {'origen': ori, 'destino': des, 'coordinates': coords}
df = pd.GeoDataFrame(d, crs=4326)
df = df.loc[df.origen != df.destino]
df.head()

In [6]:
ori = pd.DataFrame(id_diadas)[0]
des = pd.DataFrame(id_diadas)[1]

d = {'origen': ori, 'destino': des, 'geometry': lines}
df = gpd.GeoDataFrame(d, crs=4326)
df = df.loc[df.origen != df.destino]
df.head()

,origen,destino,geometry
1,1,2,"LINESTRING (-56.17428 -34.86383, -56.27460 -34..."
2,1,3,"LINESTRING (-56.17428 -34.86383, -56.17495 -34..."
3,1,4,"LINESTRING (-56.17428 -34.86383, -56.14081 -34..."
4,1,5,"LINESTRING (-56.17428 -34.86383, -56.17646 -34..."
5,1,6,"LINESTRING (-56.17428 -34.86383, -56.25299 -34..."


In [7]:
df.to_file('data/shortest_paths.geojson')

/home/guillermo/anaconda3/envs/tablero_migraciones/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [6]:
import pandas as pd
geojson_lines = pd.read_json('data/shortest_paths.geojson', orient='index')
geojson_lines

,0
type,FeatureCollection
crs,"{'type': 'name', 'properties': {'name': 'urn:o..."
features,"[{'type': 'Feature', 'properties': {'origen': ..."


In [ ]:
folium.GeoJson(antarctic_ice_edge, name="geojson").add_to(m)